# TRANSFORMACIÓN DE DATOS

## IMPORTAR PAQUETES

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

#Automcompletar rápido
%config IPCompleter.greedy=True

import warnings
warnings.filterwarnings('ignore')

## IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [2]:
ruta_proyecto = 'C:/Users/pelop/OneDrive/Desktop/Curso Data Science Pedro/2 CURSO DATA SCIENCE/03_MACHINE_LEARNING/07_CASOS/02_RETAIL/'

2.- Nombrar los ficheros de datos.

In [3]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [4]:
cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num)

## NUEVAS VARIABLES

En este caso ya nos vienen creadas:

* componentes de la fecha
* variables de calendario

Vamos a crear:

* las que identificamos en EDA
* lags
* ventanas móviles

Para esta fase necesitamos volver a unir en un solo dataframe.

In [5]:
df = pd.concat([cat,num], axis = 1)
df

,store_id,item_id,d,year,month,wday,weekday,event_name_1,event_type_1,wm_yr_wk,ventas,sell_price
date,,,,,,,,,,,,
2013-01-01,CA_3,FOODS_3_090,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,1.25
2013-01-01,CA_3,FOODS_3_120,d_704,2013,1,4,Tuesday,NewYear,National,11249,33,1.25
2013-01-01,CA_3,FOODS_3_202,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,4.98
2013-01-01,CA_3,FOODS_3_252,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,4.98
2013-01-01,CA_3,FOODS_3_288,d_704,2013,1,4,Tuesday,NewYear,National,11249,20,4.28
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-30,CA_4,FOODS_3_329,d_1767,2015,11,3,Monday,Sin_evento,Sin_evento,11544,9,1.68
2015-11-30,CA_4,FOODS_3_555,d_1767,2015,11,3,Monday,Sin_evento,Sin_evento,11544,26,2.48
2015-11-30,CA_4,FOODS_3_586,d_1767,2015,11,3,Monday,Sin_evento,Sin_evento,11544,13,2.48


### Variable demanda intermitente

Esta variable va a identificar cuantos días seguidos han transcurrido con ventas cero.

La definiremos como que si los últimos n días han tenido cero ventas entonces hay rotura de stock.

Podemos crear varias cambiando el n.

Nos servirá para modelizar.

In [12]:
def rotura_stock(ventas, n = 5):
    cero_ventas = pd.Series(np.where(ventas == 0,1,0))
    num_ceros = cero_ventas.rolling(n).sum()
    rotura_stock = np.where(num_ceros == n,1,0)
    return(rotura_stock)

In [13]:
df = df.sort_values(by = ['store_id','item_id','date'])

In [14]:
df['rotura_stock_3'] = df.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x, 3)).values

In [15]:
df['rotura_stock_7'] = df.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x,7)).values

In [16]:
df['rotura_stock_15'] = df.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x,15)).values

In [18]:
df.head(5)

,store_id,item_id,d,year,month,wday,weekday,event_name_1,event_type_1,wm_yr_wk,ventas,sell_price,rotura_stock_3,rotura_stock_7,rotura_stock_15
date,,,,,,,,,,,,,,,
2013-01-01,CA_3,FOODS_3_090,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,1.25,0,0,0
2013-01-02,CA_3,FOODS_3_090,d_705,2013,1,5,Wednesday,Sin_evento,Sin_evento,11249,224,1.25,0,0,0
2013-01-03,CA_3,FOODS_3_090,d_706,2013,1,6,Thursday,Sin_evento,Sin_evento,11249,241,1.25,0,0,0
2013-01-04,CA_3,FOODS_3_090,d_707,2013,1,7,Friday,Sin_evento,Sin_evento,11249,232,1.25,0,0,0
2013-01-05,CA_3,FOODS_3_090,d_708,2013,1,1,Saturday,Sin_evento,Sin_evento,11250,301,1.25,0,0,0


### Variables de lag

Vamos a crear lags sobre las siguientes variables:

* ventas: lags de 15 días
* sell_price: lags de 7 días
* rotura_stock: lag de un día

**Nota.-** La variable rotura_stock es mejor crearla con solo un lag de un día. Esto es debido a que los modelos de forecasting solo pueden predecir al día siguiente (al contrario que los métodos tradicionales que pueden predecir varios días). La idea es usar el método recursivo y encadenar predicciones sobre las prediciiones para acceder a días posteriores. La particularidad de la variable rotura_stock está en que es muy raro que el modelo nos vaya a predecir exactamente 0, puede predecir valores muy bajos pero exactamente 0 sería muy raro y la variable no funcionaría bien (de acuerdo a como la hemos definido nosotros). Así pues, solo el primer lag utilizaría un dato real y los siguientes sería sobre los predecidos. De ahí que solo vayamos a usar lags de 1 día para ella.

In [19]:
def crear_lags(df, variable, num_lags = 7):
    
    #Crea el objeto dataframe
    lags = pd.DataFrame()
    
    #Crea todos los lags
    for cada in range(1,num_lags+1):
        lags[variable + '_lag_'+ str(cada)] = df[variable].shift(cada)
    
    #Devuelve el dataframe de lags
    return(lags)

In [22]:
lags_sell_price_df = (df.groupby(['store_id', 'item_id'])
                        .apply(lambda x: crear_lags(df = x, variable = 'sell_price', num_lags= 7))
                        .reset_index()
                        .set_index('date'))


In [23]:
lags_rotura_stock_3_df = (df.groupby(['store_id','item_id'])
                            .apply(lambda x: crear_lags(df = x, variable = 'rotura_stock_3', num_lags= 1))
                            .reset_index()
                            .set_index('date'))

In [24]:
lags_rotura_stock_7_df = (df.groupby(['store_id','item_id'])
                            .apply(lambda x: crear_lags(df = x, variable = 'rotura_stock_7', num_lags= 1))
                            .reset_index()
                            .set_index('date'))

In [25]:
lags_rotura_stock_15_df = (df.groupby(['store_id','item_id'])
                            .apply(lambda x: crear_lags(df = x, variable = 'rotura_stock_15', num_lags= 1))
                            .reset_index()
                            .set_index('date'))

In [26]:
lags_ventas_df = (df.groupby(['store_id','item_id'])
                    .apply(lambda x: crear_lags(df = x, variable = 'ventas', num_lags= 15))
                    .reset_index()
                    .set_index('date'))

### Variables de ventanas móviles

Vamos a crear tres tipos de ventanas móviles sobre las ventas:

* mínimo móvil
* media móvil
* máximo móvil

Cada uno de ellos en el rango de 15 días.

**Nota.-** Recordar que usamos el *shift(1)* para retrasar los datos un día hacia atrás. Para predecir el día actual necesito los datos de ayer, no los de hoy que aún no los tenemos.

In [30]:
def min_movil(df, variable, num_periodos = 7):

    minm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        minm[variable + '_minm_' + str(cada)] = df[variable].shift(1).rolling(cada).min()
    
    #Devuelve el dataframe de lags
    return(minm)

In [31]:
def media_movil(df, variable, num_periodos = 7):

    mm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        mm[variable + '_mm_' + str(cada)] = df[variable].shift(1).rolling(cada).mean()
    
    #Devuelve el dataframe de lags
    return(mm)

In [32]:
def max_movil(df, variable, num_periodos = 7):

    maxm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        maxm[variable + '_maxm_' + str(cada)] = df[variable].shift(1).rolling(cada).max()
    
    #Devuelve el dataframe de lags
    return(maxm)

In [33]:
min_movil_df = (df.groupby(['store_id','item_id'])
                  .apply(lambda x: min_movil(df = x, variable = 'ventas', num_periodos= 15))
                  .reset_index()
                  .set_index('date'))

In [34]:
media_movil_df = (df.groupby(['store_id','item_id'])
                    .apply(lambda x: media_movil(df = x, variable = 'ventas', num_periodos= 15))
                    .reset_index()
                    .set_index('date'))

In [35]:
max_movil_df = (df.groupby(['store_id','item_id'])
                    .apply(lambda x: max_movil(df = x, variable = 'ventas', num_periodos= 15))
                    .reset_index()
                    .set_index('date'))

## PREPARAR LOS DATASETS

### Unir todos los dataframes generados

In [42]:
df_unido = pd.concat([df,
                      lags_sell_price_df,
                      lags_rotura_stock_3_df,
                      lags_rotura_stock_7_df,
                      lags_rotura_stock_15_df,
                      lags_ventas_df,
                      min_movil_df,
                      media_movil_df,
                      max_movil_df], axis = 1)

# Eliminar columnas duplicadas
df_unido = df_unido.loc[:,~df_unido.columns.duplicated()]
df_unido

,store_id,item_id,d,year,month,wday,weekday,event_name_1,event_type_1,wm_yr_wk,...,ventas_maxm_6,ventas_maxm_7,ventas_maxm_8,ventas_maxm_9,ventas_maxm_10,ventas_maxm_11,ventas_maxm_12,ventas_maxm_13,ventas_maxm_14,ventas_maxm_15
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,CA_3,FOODS_3_090,d_704,2013,1,4,Tuesday,NewYear,National,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,CA_3,FOODS_3_090,d_705,2013,1,5,Wednesday,Sin_evento,Sin_evento,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,CA_3,FOODS_3_090,d_706,2013,1,6,Thursday,Sin_evento,Sin_evento,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,CA_3,FOODS_3_090,d_707,2013,1,7,Friday,Sin_evento,Sin_evento,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-05,CA_3,FOODS_3_090,d_708,2013,1,1,Saturday,Sin_evento,Sin_evento,11250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-26,CA_4,FOODS_3_714,d_1763,2015,11,6,Thursday,Thanksgiving,National,11543,...,15.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0
2015-11-27,CA_4,FOODS_3_714,d_1764,2015,11,7,Friday,Sin_evento,Sin_evento,11543,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
2015-11-28,CA_4,FOODS_3_714,d_1765,2015,11,1,Saturday,Sin_evento,Sin_evento,11544,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0


### Eliminar los nulos que han generado las nuevas variables

In [43]:
df_unido.dropna(inplace=True)

In [45]:
df_unido.isna().sum().sort_values(ascending=False)

store_id                 0
ventas_mm_9              0
ventas_mm_7              0
ventas_mm_6              0
ventas_mm_5              0
                        ..
ventas_lag_2             0
ventas_lag_1             0
rotura_stock_15_lag_1    0
rotura_stock_7_lag_1     0
ventas_maxm_15           0
Length: 82, dtype: int64

### Eliminar las variables que no vamos a necesitar para modelizar

Eliminamos las variables que no vamos a usar y las originales de las que se han derivado los lags.

In [46]:
a_eliminar = ['d','wm_yr_wk','sell_price','rotura_stock_3','rotura_stock_7','rotura_stock_15']

In [47]:
df_unido.drop(columns=a_eliminar, inplace=True)

### Identificar la target

In [48]:
target = df_unido.ventas

### Separar num y cat

In [49]:
cat = df_unido.select_dtypes(include='O')

In [50]:
num = df_unido.select_dtypes(exclude='O')

## TRANSFORMACIÓN DE CATEGÓRICAS

### One Hot Encoding

#### Variables a aplicar OHE

In [51]:
var_ohe = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

#### Instanciar

In [52]:
ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')

#### Entrenar y aplicar

In [53]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Guardar como dataframe

In [54]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

### Target Encoding

#### Variables a aplicar TE

In [55]:
var_te = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

#### Instanciar

In [56]:
te = TargetEncoder(min_samples_leaf=100, return_df = False)

#### Entrenar y aplicar

In [57]:
cat_te = te.fit_transform(cat[var_te], y = target)

#### Guardar como dataframe

In [58]:
#Añadir sufijos a los nombres
nombres_te = [variable + '_te' for variable in var_te]

#Guardar como dataframe
cat_te = pd.DataFrame(cat_te, columns = nombres_te)

## UNIFICAR DATASETS TRANSFORMADOS

### Meter en una lista todos los dataframes generados

Rescatamos de df_unido las variables de segmentación.

In [59]:
de_df_unido = df_unido[['store_id','item_id']].reset_index()

de_df_unido.head(2)

,date,store_id,item_id
0,2013-01-16,CA_3,FOODS_3_090
1,2013-01-17,CA_3,FOODS_3_090


### Unir todos los dataframes

In [60]:
dataframes = [de_df_unido, cat_ohe,cat_te,num.reset_index(drop=True)]

In [61]:
df_tablon = pd.concat(dataframes, axis = 1)

df_tablon

,date,store_id,item_id,year_2013,year_2014,year_2015,month_1,month_2,month_3,month_4,...,ventas_maxm_6,ventas_maxm_7,ventas_maxm_8,ventas_maxm_9,ventas_maxm_10,ventas_maxm_11,ventas_maxm_12,ventas_maxm_13,ventas_maxm_14,ventas_maxm_15
0,2013-01-16,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
1,2013-01-17,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
2,2013-01-18,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
3,2013-01-19,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,281.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
4,2013-01-20,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20975,2015-11-26,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0
20976,2015-11-27,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
20977,2015-11-28,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
20978,2015-11-29,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0


## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [62]:
#Definir los nombres del archivo
ruta_df_tablon = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [63]:
#Guardar los archivos
df_tablon.to_pickle(ruta_df_tablon)